# Column Generation in Cutting Stock Problem

# 1. Cutting Stock Problem (CSP)


## 1.1. MIP
La formulción MIP del CSP es la siguiente:

\begin{equation*}
\begin{array}
& & \underset{x}{\text{minimize}}
& & \sum_{j=1}^{n}x_{j} \\
& \text{subject to}
& & \sum_{j=1}^{n}y_{ij} \geq d_{i}, \; i = 1, \ldots, m,\\
& & &\sum_{i=1}^{m}l_{i}y_{ij} \leq Lx_{j}, \; j = 1, \ldots, n,\\
& & & y_{ij} \in \mathbb{Z_{+}}, \; x_{j} \in \{0,1\}, \; i = 1, \ldots, m, \; j=1, \ldots n.\\
\end{array}
\end{equation*}

Donde $n$ es el número de stocks en inventario listos para ser cortados y $m$ el número de items diferentes que se puede obtener de un stock, $l_i$ es el largo del items $i$ y $L$ el largo del stock, $y_{ij}$ corresponde al número de items del tipo $i$ obtenidos al cortar el stock $j$, $x_{j}$ es una variable binaria que indica si el stock $j$ es cortado o no. La primera restricción obliga a satisfacer la demanda por cada items, la segunda restricción limita la cantidad de items que se pueden generar de un stock, donde la suma de los largos de los productos generados a partir de un stock no puede superar su largo, además, implica que si se genera un item a partir de una stock esta debe ser cortado. La función objetivo es minimizar el número de stock a ser cortados. 

## 1.2. Problema Maestro (MP)

\begin{equation*}
\begin{array}
& & \underset{\lambda}{\text{minimize}}
& & \sum_{k=1}^{K}\lambda_{k} \\
& \text{subject to}
& & \sum_{k=1}^{K}y_{ik}\lambda_{k} \geq d_{i}, \; i = 1, \ldots, m,\\
& & &\sum_{k=1}^{K}\lambda_{k} \leq n,\\
& & & \lambda_{k}\in \mathbb{Z_{+}}, \; k = 1, \ldots, K.\\
\end{array}
\end{equation*}

Donde $K$ es el número de formas factibles de cortar el stock (o número de patrones), $y_{ik}$ es el número de items de la clase $i$ generados en el corte o patrón $k$, $\lambda_{k}$ es el número de patrónes del tipo $k$ a cortar, cada patrón tiene asociado un vector $y_{k}=\{y_{1k},\ldots, y_{mk}\}$. La primera restricción obliga a satisfacer la demanda por cada items, la segunda restricción obliga a que el número total de patrones cortados sea inferior al número de stocks disponibles. La función objetivo es equivalente a la formulación anterior, ya que cada $\lambda_{k}$ corresponde al número de stock cortados en el patrón $k$ y al sumar sobre $K$ se obtien el total de stock cortados que es el valor que se desea minimizar.

## 1.3 Column Generation (CG)

Como el número de columnas que contiene el RP puede ser un número gigante, es bastante frecuente trabajar con una versión restringida del MP, el cual es llamado el Restricted Master Problem (RMP), el cual contiene un subconjunto de las columnas del MP y genera columnas adicionales solo cuando estas son necesarias. En el proceso generativo se resuelve el LP del RMP, luego con las variables duales de la solución resultante se resuelve el Sub Problema (SP) también llamado Pricing Problema, problema cuya función objetivo corresponde al costo reducido del RMP en función de un patrón y el objetivo es encontrar alguna columna o patrón $y$ con costo reducido negativo resolviendo un problema de optimización, si la columna generada tiene costo reducido negativo entonces esta entra a la base del RMP, si no, entonces la actual solución del RMP es además óptima para el MP y términa el proceso de generación de columnas.


Para generar columnas en el nodo raíz es necesario contar con una cantidad suficiente de columnas para generar una base, para esto se escogerá una matriz diagonal, donde cada elemento de la diagonal representa la cantidad máxima de items de un tipo que se puede obtener del stock, esto es $\lfloor L/l_{i} \rfloor$.

## 1.4 Sub-Problema (SP)

\begin{equation*}
\begin{array}
& & \underset{x,y}{\text{minimize}}
& & [x-\pi^{T}y]+\alpha \\
& \text{subject to}
& & \sum_{i=1}^{m}l_{i}y_{i} \leq Lx_,\\
& & & y_{i} \in \mathbb{Z_{+}}, \; x \in \{0,1\}, \; i = 1, \ldots, m.\\
\end{array}
\end{equation*}

Donde $\pi$ corresponde a las variables duales asociadas a la primera restricción del RMP y $\alpha$ la variable dual asociada a la segunda restricción, como $\alpha$ no tiene asociada ninguna variable de decisión se puede omitir del problema, además, como el objetivo es encontrar columnas con costo reducido negativo podemos eliminar la variable $x$ del problema, ya que si fijamos $x=0$ el costo reducido es 0, por tanto, fijando $x=1$ se tiene el siguiente subproblema:

\begin{equation*}
\begin{array}
& & \underset{x,y}{\text{max}}
& & \pi^{T}y \\
& \text{subject to}
& & \sum_{i=1}^{m}l_{i}y_{i} \leq L,\\
& & & y_{i} \in \mathbb{Z_{+}}, \; i = 1, \ldots, m.\\
\end{array}
\end{equation*}

Notar que el subproblema es un problema de la mochila (Knapsack Problem), siendo el $c = 1-\pi^{T}y+\alpha $ el costo reducido.

## 1.5. Que cotas puede demostrar (para el problema global) si interrumpe la generacion de columnas en el nodo raiz cuando el problema de ’pricing’ retorna un valor -e?


#### Cota inferior
Para el problema de cutting stock están las cotas triviales, que para la cota inferior es el óptimo del problema relajado con patrones fraccionales, es decir que a una tabla le puedo asignar un resultado fraccional de algún corte. Esta cota, por construcción, es cota inferior del problema con la relajación lagrangeana del primal, donde sea $B_{fra}$ la función objetivo en el óptimo del probleman fraccional y $B_{lprmp}$ el óptimo de la relajación del problema maestro:
$$ B_{lprm} \geq B_{frac} $$

Es fácil notar que esta cota trivial es poco estrecha, y debido a que es cota inferior del problema maestro relajado, es irrelevante en este caso debido a que se tiene una cota inferior conocida para este tipo de problema en "Branch-and-Price Algorithms..."[3], donde:

Sea $z_{B}$ el óptimo del problema maestro detenido en el nodo raíz en el cual el costo reducido de la última variable agregada sea $-\epsilon$ y $z_{IP}$ el óptimo del problema maestro original, entonces siempre y cuando $\lceil z_B \rceil = \lfloor\frac{z_b}{(1-\epsilon)}\rfloor$, entonces $z_{IP} \geq \lceil z_b \rceil $

Lo que se demuestra usando el resultado de Farley[4] donde se demuestra que: $ z_{min} \geq \frac{z_b}{1- \epsilon} $

Utilizando este resultado junto con $z_B \geq z_{min}$ se obtiene qué: $ \lceil z_B \rceil = \lceil \frac{z_B}{1-\epsilon} \rceil $ donde usando que $\lceil z_{min} \rceil = \lceil z_B \rceil $ se concluye que $z_{IP} \geq \lceil z_B \rceil $ que es la cota mencionada anteriormente."Branch-and-Price Algorithms..."[3]

El paper también destaca que esta cota permite identificar una regla para determinar el punto de branching, donde no conviene seguir resolviendo el problema el LP.

#### Cota superior 
Para la cota superior solo podemos decir que la cota trivial es la solción del problema maestro con la solución básica factible trivial, donde cada patrón de corte es aquel donde cada tabla tiene exactamente un corte de cada tipo, donde hay estrictamente la misma cantidad de patrones como cortes hay.

## 2 Código

In [1]:
import gurobipy as grb
import numpy as np
import pandas as pd

In [2]:
#Master Problem del CSP
def CSP_MP(n, d, y, Gap, time, verbose = False):
    """
    Cutting Stock Problem - Master Problem
    Inputs:
    
    1. n: int, número de stock disponibles.
    2. d: numpy array, vector con las demandas de los m items.
    3. y: numpy array, matriz de m filas y K columnas con las formas factibles de cortar el stock.
    4. Gap: float, Gap entre UB y LB.
    5. time: float, tiempo límite de ejecución.
    """
    model = grb.Model('CSP-MP')
    model.setParam('OutputFlag', verbose)
    
    m, K = y.shape
    
    ## Variables
    #lambda_{k} es el número de patrónes del tipo k a cortar
    lamb = model.addVars(range(K), vtype='I', lb=0,name='lambda')
    
    ## Restricciones
    #1. Satisfacer la demanda por cada items
    model.addConstrs(grb.quicksum(y[i,k]*lamb[k] for k in range(K))>=d[i] for i in range(m))
    #2. El número total de patrones cortados es igual o menor al número de stocks disponibles
    model.addConstr(grb.quicksum(lamb[k] for k in range(K))<=n)
    
    ##Función objetivo
    model.setObjective(grb.quicksum(lamb[k] for k in range(K)))
    
    model.update()
    #Sentido de la optimización
    model.ModelSense = grb.GRB.MINIMIZE
    #Setear GAP y optimizar
    model.Params.MIPGap = Gap
    #Número de hilos a usar
    model.setParam(grb.GRB.Param.Threads, 4)
    #Tolerancia de infactibilidad de una variable binaria para evitar soluciones fraccionarias
    model.Params.IntFeasTol = 1e-9 
    #Limite de tiempo para problemas que demoren en correr
    model.Params.TIME_LIMIT= time
    
    model.optimize()
    
    return model
    

In [3]:
#Relajación lineal del RMP del CSP
def CSP_LPRMP(n, d, y, Gap, time, lb=None, ub=None, verbose = False):
    """
    Cutting Stock Problem - Linear Problem relaxation of Restricted Master Problem
    Inputs:
    
    1. n: int, número de stock disponibles.
    2. d: numpy array, vector con las demandas de los m items.
    3. y: numpy array, matriz de m filas y K columnas con las formas factibles de cortar el stock.
    4. Gap: float, Gap entre UB y LB.
    5. time: float, tiempo límite de ejecución.
    6. lb: list, lista con las cotas inferiores de las variables.
    7. ub: list, lista con las cotas superiores de las variables.
    8. verbose: bool, si es True se muestran los logs del solver.
    """
    model = grb.Model('CSP-LPRMP')
    model.setParam('OutputFlag', verbose)
    m, K = y.shape
    
    ## Variables
    #lambda_{k} es el número de patrónes del tipo k a cortar
    lamb = model.addVars(range(K), vtype='C', lb=0, name='lambda')
    #actualizar cota inferior y superior de las variables
    if (lb!=None) and (ub!=None):
        for i in range(K):
            lamb[i].lb= lb[i]
            lamb[i].ub = ub[i]
    ## Restricciones
    #1. Satisfacer la demanda por cada items
    model.addConstrs(grb.quicksum(y[i,k]*lamb[k] for k in range(K))>=d[i] for i in range(m))
    #2. El número total de patrones cortados es igual o menor al número de stocks disponibles
    model.addConstr(grb.quicksum(lamb[k] for k in range(K))<=n)
    
    ##Función objetivo
    model.setObjective(grb.quicksum(lamb[k] for k in range(K)))
    
    model.update()
    #Sentido de la optimización
    model.ModelSense = grb.GRB.MINIMIZE
    #Setear GAP y optimizar
    model.Params.MIPGap = Gap
    #Número de hilos a usar
    model.setParam(grb.GRB.Param.Threads, 4)
    #Tolerancia de infactibilidad de una variable binaria para evitar soluciones fraccionarias
    model.Params.IntFeasTol = 1e-9 
    #Limite de tiempo para problemas que demoren en correr
    model.Params.TIME_LIMIT= time
    
    model.optimize()
    
    return model
    

In [4]:
def CSP_SP(L, l, pi, Gap, time, verbose = False):
    
    """
    Cutting Stock Probelm - Sub Problem
    Inputs:
    1. L: int, largo del stock.
    2. l: numpy array, vector con los largos de los items en int.
    3. pi: numpy array, vector con las variables duales de la primera restricción del RMP.
    4. Gap: float, Gap entre UB y LB.
    5. time: float, tiempo límite de ejecución.
    """
    model = grb.Model('CSP-SP')
    model.setParam('OutputFlag', verbose)
    m = len(pi)
        
    ## Variables
    # y_{i} es el número de items i a cortar
    y = model.addVars(range(m), vtype='I', lb=0,  name='y')
    
    ## Restricciones
    # La suma de los largos de los productos generados a partir de un stock no puede superar su largo
    model.addConstr(grb.quicksum(l[i]*y[i] for i in range(m))<=L)
        
    ##Función objetivo
    model.setObjective(grb.quicksum(pi[i]*y[i] for i in range(m)))
    
    model.update()
    
    #Sentido de la optimización
    model.ModelSense = grb.GRB.MAXIMIZE
    #Setear GAP y optimizar
    model.Params.MIPGap = Gap
    #Número de hilos a usar
    model.setParam(grb.GRB.Param.Threads, 4)
    #Tolerancia de infactibilidad de una variable binaria para evitar soluciones fraccionarias
    model.Params.IntFeasTol = 1e-9 
    #Limite de tiempo para problemas que demoren en correr
    model.Params.TIME_LIMIT= time
    
    model.optimize()
    
    return model
    
    

In [5]:
def CSP_IS(L,l):
    
    """
    Cutting Stock Problem -  Initial Solution of Column Generation
    Entrega solución inicial para CSP_CG
    Inputs:
    1. L: float, largo del stock.
    2. l: numpy array, vector con los largos de los items.
    
    Output:
    y: np.array, matriz con m maximal cutting patterns
    """
    #Solución inicial del CSP
    
    #Inicialización de los patrones de corte con una matriz diagonal,
    #cada elemento de la diagonal representa la cantidad máxima de items de un tipo que se puede obtener del stock
    y = np.diag((L/l).astype(int))

    i_min, l_min = l.argmin(), l.min()
    
    #Volvemos cada columna un maximal, es decir, el sobrante es menor al item más corto
    #por ende rellenamos con el item más corto hasta que no quede espacio para ningún item
    for i in range(y.shape[1]):
        waste = L-(l*y[:, i]).sum()
        n_l_min = int(waste/l_min)
        y[i_min, i] =  y[i_min, i]+n_l_min
        
    return y

In [6]:
def CSP_CG(L, l, n, d, Gap, time, lb=None, ub=None, y_init=None):
    """
    Cutting Stock Problem - Column Generation
    Inputs:
    1. L: float, largo del stock.
    2. l: numpy array, vector con los largos de los items.
    3. n: int, número de stock disponibles.
    4. d: numpy array, vector con las demandas de los m items.
    5. Gap: float, Gap entre UB y LB.
    6. time: float, tiempo límite de ejecución.
    7 lb: list, lista con las cotas inferiores de las variables del LPRMP
    8 ub: list, lista con las cotas superiores de las variables del LPRMP
    9. y_init: numpy array, columnas iniciales.
    """
    m = len(l)
    
    #Inicialización,  matriz con m maximal cutting patterns si es que no se  le han sumistrado columnas
    if y_init is None:
        y = CSP_IS(L,l)
    else:
        y=y_init
        
    #Costo reducido inicial
    c=float("-inf")
    #Lista donde se alamacenará los costos reducidos
    reduce_costs = []
    i = 0
    while (c<0):
        i = i+1
        #Resolver LP del RMP
        print('Iteración {i} de la generación de columnas'.format(i=i), '\n')
        print('  Resolviendo el LPRMP en la iteración {i}'.format(i=i), '\n')
        rmp = CSP_LPRMP(n, d, y, Gap, time, lb, ub) 
        #Obtener las variables duales
        pi = np.array(rmp.Pi[:-1])
        #Resolver SP
        print('  Resolviendo el SP en la iteración {i}'.format(i=i), '\n')
        sp = CSP_SP(L, l, pi, Gap, time)
        #Actualizar costos reducidos
        c = 1-sp.objVal+rmp.Pi[-1]
        reduce_costs.append(c)
        
        #Añadir columna si el costo reducido de la columna es negativo
        if (c<0):
            #Nueva columna
            y_k = np.array(sp.X).reshape(m,1)
            #Añadir
            y = np.append(y, y_k, axis=1)
        if (lb!=None) and (ub!=None):
            lb.append(0)
            ub.append(grb.GRB.INFINITY)
    return y

In [7]:
def CSP_RI(m, n, seed):
    """
    Cutting Stock Problem - Random Instance Generator
    Genera una instancia aleatoria del Cutting Stock Problem
    
    Inputs
    1. m: int, número de items.
    2. n: int, número de stocks disponibles.
    3. seed: int, semilla aleatoria.
    
    Outpus
    1. l: numpy array, vector con los largos en int de los items.
    2. d: numpy array, vector con 
    3. L: int, largo del stock.
    """
    np.random.seed(seed)
    l = np.random.randint(low=1, high=100, size=m)
    d = np.random.randint(low=10, high=100, size=m)
    #En el LP del CSP el largo debe cumplir L>=l'd/n, además debe ser más largo que el item más largo
    L= max(4*int((l*d).sum()/n), l.max()) 
    return l, d, L

# 3. Instancias

Resolviendo el problema de generación de columnas en el nodo raíz y luego branch and bound

In [10]:
#Instancia
m, n, seed = 10, 100, 1
l, d, L = CSP_RI(m, n, seed)
Gap, time = 0, 1e10

In [11]:
#Resolver instancia
y = CSP_CG(L, l, n, d, Gap, time)
mp = CSP_MP(n, d, y, Gap, time, verbose = False)

Iteración 1 de la generación de columnas 

  Resolviendo el LPRMP en la iteración 1 

Academic license - for non-commercial use only
  Resolviendo el SP en la iteración 1 

Iteración 2 de la generación de columnas 

  Resolviendo el LPRMP en la iteración 2 

  Resolviendo el SP en la iteración 2 

Iteración 3 de la generación de columnas 

  Resolviendo el LPRMP en la iteración 3 

  Resolviendo el SP en la iteración 3 

Iteración 4 de la generación de columnas 

  Resolviendo el LPRMP en la iteración 4 

  Resolviendo el SP en la iteración 4 

Iteración 5 de la generación de columnas 

  Resolviendo el LPRMP en la iteración 5 

  Resolviendo el SP en la iteración 5 

Iteración 6 de la generación de columnas 

  Resolviendo el LPRMP en la iteración 6 

  Resolviendo el SP en la iteración 6 

Iteración 7 de la generación de columnas 

  Resolviendo el LPRMP en la iteración 7 

  Resolviendo el SP en la iteración 7 

Iteración 8 de la generación de columnas 

  Resolviendo el LPRMP en la

In [12]:
"""Resultados, cada fila es una columna (patrón) y cada clumna es un item, 
la última columna corresponde a las variables de decisión"""

df = pd.DataFrame(np.append(y.T, np.array(mp.X).reshape(len(mp.X),1), axis=1), columns=list(range(len(y)))+['lambda'])
df

,0,1,2,3,4,5,6,7,8,9,lambda
0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,4.0
1,0.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-0.0
2,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,-0.0
3,0.0,0.0,0.0,77.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,6.0,-0.0
5,0.0,0.0,0.0,0.0,0.0,128.0,0.0,0.0,0.0,2.0,-0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,26.0,1.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,28.0,-0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,3.0,-0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,386.0,-0.0


In [13]:
print('Total de stock cortados:', mp.objVal)
print('LB de stock cortados viene dada por el LP del MP:', (l*d).sum()/L)

Total de stock cortados: 27.0
LB de stock cortados viene dada por el LP del MP: 25.080310880829014


In [14]:
#Instancia
m, n, seed = 2, 10, 1
l, d, L = CSP_RI(m, n, seed)
Gap, time = 0, 1e10

In [15]:
#Resolver instancia
y = CSP_CG(L, l, n, d, Gap, time)
mp = CSP_MP(n, d, y, Gap, time, verbose = False)

Iteración 1 de la generación de columnas 

  Resolviendo el LPRMP en la iteración 1 

  Resolviendo el SP en la iteración 1 

Iteración 2 de la generación de columnas 

  Resolviendo el LPRMP en la iteración 2 

  Resolviendo el SP en la iteración 2 

Iteración 3 de la generación de columnas 

  Resolviendo el LPRMP en la iteración 3 

  Resolviendo el SP en la iteración 3 



In [1018]:
"""Resultados, cada fila es una columna (patrón) y cada clumna es un item, 
la última columna corresponde a las variables de decisión"""

df = pd.DataFrame(np.append(y.T, np.array(mp.X).reshape(len(mp.X),1), axis=1), columns=list(range(len(y)))+['lambda'])
df

,0,1,lambda
0,35.0,1.0,3.0
1,0.0,103.0,-0.0
2,8.0,80.0,1.0
3,34.0,4.0,-0.0


In [1019]:
print('Total de stock cortados:', mp.objVal)
print('LB de stock cortados viene dada por el LP del MP:', (l*d).sum()/L)

Total de stock cortados: 4.0
LB de stock cortados viene dada por el LP del MP: 2.502232142857143
